In [1]:
import sys
sys.path.append('../../ml_utils')

import config as cfg
import data_utils as du
import autoencoder as aenc
import numpy as np
import pandas as pd
import timeit
import matplotlib.pyplot as plt
import joblib
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
from keras import regularizers

%matplotlib inline

Using TensorFlow backend.
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [2]:
print("joblib (0.14.1): {}\n".format(joblib.__version__))
print("numpy (1.17.4): {}\n".format(np.__version__))
print("pandas (0.25.3): {}\n".format(pd.__version__))
print("sklearn (0.22): {}\n".format(sklearn.__version__))

joblib (0.14.1): 0.14.1

numpy (1.17.4): 1.17.4

pandas (0.25.3): 0.25.3

sklearn (0.22): 0.22



### Load and clean the data

In [3]:
log_data = './../../shared/data/swissid_authorize_logs_april_to_sept_2019.csv'

start = timeit.default_timer()
df = pd.read_csv(filepath_or_buffer=log_data, header=0, sep='\t', names=cfg.complete_feature_list, index_col=None)
stop = timeit.default_timer()

print("Original data frame size: {}\n".format(df.shape))
print("Time: {} seconds\n".format(stop - start))

Original data frame size: (12417597, 31)

Time: 39.523410859999785 seconds



In [4]:
df = du.clean_data(df)

columns with NaN: loc_country_code
loc_country_code is fixed


In [5]:
features = ['label_nr', 'src_software_sub_type', 'src_operating_system_name', 'src_hardware_type', 'response_status_code', 
            'oidc_client_id', 'oidc_scopes', 'oidc_ui_locales', 'loc_city', 'loc_country_code', 'date_weekday']

excludes = list(set(df.columns.tolist()) - set(features))

In [6]:
reduced_df = du.reduce_features(df, excludes)

print("reduced data frame size: {}\n".format(reduced_df.shape))
#display(reduced_df)

reduced data frame size: (12204748, 11)



In [7]:
print(reduced_df['label_nr'].value_counts())

2    7412813
0    4769169
1      22766
Name: label_nr, dtype: int64


### Build a sampled dataframe having the same anomaly rate as the original dateset and collect the metrics data


In [10]:
iterations = 100
metrics = []
iter_values = []

In [11]:
sample_size= 300000
exclude = ['label_nr']

batch_size = 32
epochs = 25
optimizer='Adamax'
encoding_dim = 4

start_overall = timeit.default_timer()

for i in range(iterations):

    X_df, y_df, anomaly_rate, encoder = du.build_normal_anomaly_sample_df(reduced_df, sample_size, exclude, True, True, False, True)
    x_train, x_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=42)

    
    model = aenc.create_sparse_auto_encoder(x_train.shape[1], encoding_dim)

    
    start_fitting = timeit.default_timer()
    history = aenc.auto_encoder_fit(model, x_train, x_test, optimizer, epochs, batch_size)
    stop_fitting = timeit.default_timer()
    
    fit_time = stop_fitting - start_fitting
    print("{0} Time to fit: {1:.2f} seconds".format(i, fit_time))
    
    
    y_pred = model.predict(x_test)
    thres, f1, p, r, sup, acc = aenc.calc_metrics(x_test, y_test, y_pred)
    y_class, errors = aenc.classify(x_test, y_pred, thres)
    auc = roc_auc_score(y_test, y_class)
    tn, fp, fn, tp = confusion_matrix(y_test, y_class).ravel()
    
    iter_values = [len(y_test),thres,r,p,acc,f1,tp,fp,fn,tn,auc,fit_time]
        
    metrics.append(iter_values)
    
    du.create_autoencoder_metric_df(metrics, './autoencoder_metrics.csv')
    
    iter_values = []

stop_overall = timeit.default_timer()

overall_runtime = stop_overall - start_overall
print("Time: {0:.2f} seconds\n".format(overall_runtime))
print("Time: {0:.2f} minutes\n".format(overall_runtime/60))

Train on 210000 samples, validate on 90000 samples
Epoch 1/25
210000/210000 [==============================] - 20s 96us/step - loss: 0.8840 - mean_absolute_error: 0.4998 - acc: 0.6730 - val_loss: 0.8541 - val_mean_absolute_error: 0.4888 - val_acc: 0.8327
Epoch 2/25
210000/210000 [==============================] - 20s 94us/step - loss: 0.8440 - mean_absolute_error: 0.4870 - acc: 0.8437 - val_loss: 0.8439 - val_mean_absolute_error: 0.4826 - val_acc: 0.8438
Epoch 3/25
210000/210000 [==============================] - 20s 94us/step - loss: 0.8364 - mean_absolute_error: 0.4790 - acc: 0.8808 - val_loss: 0.8390 - val_mean_absolute_error: 0.4764 - val_acc: 0.9010
Epoch 4/25
210000/210000 [==============================] - 20s 94us/step - loss: 0.8333 - mean_absolute_error: 0.4761 - acc: 0.8959 - val_loss: 0.8370 - val_mean_absolute_error: 0.4750 - val_acc: 0.9083
Epoch 5/25
210000/210000 [==============================] - 20s 95us/step - loss: 0.8317 - mean_absolute_error: 0.4753 - acc: 0.9136 

/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 210000 samples, validate on 90000 samples
Epoch 1/25
210000/210000 [==============================] - 20s 97us/step - loss: 0.8968 - mean_absolute_error: 0.4912 - acc: 0.5104 - val_loss: 0.8956 - val_mean_absolute_error: 0.4819 - val_acc: 0.6215
Epoch 2/25
210000/210000 [==============================] - 20s 96us/step - loss: 0.8609 - mean_absolute_error: 0.4782 - acc: 0.6339 - val_loss: 0.8865 - val_mean_absolute_error: 0.4782 - val_acc: 0.6351
Epoch 3/25
210000/210000 [==============================] - 20s 95us/step - loss: 0.8562 - mean_absolute_error: 0.4766 - acc: 0.6397 - val_loss: 0.8837 - val_mean_absolute_error: 0.4776 - val_acc: 0.6391
Epoch 4/25
210000/210000 [==============================] - 20s 96us/step - loss: 0.8541 - mean_absolute_error: 0.4767 - acc: 0.6391 - val_loss: 0.8819 - val_mean_absolute_error: 0.4783 - val_acc: 0.6357
Epoch 5/25
210000/210000 [==============================] - 20s 95us/step - loss: 0.8505 - mean_absolute_error: 0.4778 - acc: 0.6528 

/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 210000 samples, validate on 90000 samples
Epoch 1/25
210000/210000 [==============================] - 21s 99us/step - loss: 0.8979 - mean_absolute_error: 0.5001 - acc: 0.5965 - val_loss: 0.8440 - val_mean_absolute_error: 0.4807 - val_acc: 0.8076
Epoch 2/25
210000/210000 [==============================] - 21s 100us/step - loss: 0.8443 - mean_absolute_error: 0.4760 - acc: 0.8634 - val_loss: 0.8319 - val_mean_absolute_error: 0.4734 - val_acc: 0.8837
Epoch 3/25
210000/210000 [==============================] - 21s 99us/step - loss: 0.8391 - mean_absolute_error: 0.4734 - acc: 0.8850 - val_loss: 0.8284 - val_mean_absolute_error: 0.4724 - val_acc: 0.8863
Epoch 4/25
210000/210000 [==============================] - 21s 99us/step - loss: 0.8358 - mean_absolute_error: 0.4728 - acc: 0.9145 - val_loss: 0.8254 - val_mean_absolute_error: 0.4720 - val_acc: 0.9223
Epoch 5/25
210000/210000 [==============================] - 21s 100us/step - loss: 0.8335 - mean_absolute_error: 0.4723 - acc: 0.930

...

Train on 210000 samples, validate on 90000 samples
Epoch 1/25
210000/210000 [==============================] - 33s 155us/step - loss: 0.8815 - mean_absolute_error: 0.5116 - acc: 0.4982 - val_loss: 0.8596 - val_mean_absolute_error: 0.4886 - val_acc: 0.7436
Epoch 2/25
210000/210000 [==============================] - 30s 144us/step - loss: 0.8291 - mean_absolute_error: 0.4813 - acc: 0.8404 - val_loss: 0.8455 - val_mean_absolute_error: 0.4801 - val_acc: 0.8669
Epoch 3/25
210000/210000 [==============================] - 30s 144us/step - loss: 0.8231 - mean_absolute_error: 0.4777 - acc: 0.8749 - val_loss: 0.8424 - val_mean_absolute_error: 0.4786 - val_acc: 0.8788
Epoch 4/25
210000/210000 [==============================] - 30s 144us/step - loss: 0.8207 - mean_absolute_error: 0.4770 - acc: 0.8838 - val_loss: 0.8405 - val_mean_absolute_error: 0.4784 - val_acc: 0.8786
Epoch 5/25
210000/210000 [==============================] - 30s 144us/step - loss: 0.8192 - mean_absolute_error: 0.4767 - acc: 0.

/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 210000 samples, validate on 90000 samples
Epoch 1/25
210000/210000 [==============================] - 33s 155us/step - loss: 0.8789 - mean_absolute_error: 0.4952 - acc: 0.6737 - val_loss: 0.9027 - val_mean_absolute_error: 0.4811 - val_acc: 0.8127
Epoch 2/25
210000/210000 [==============================] - 30s 144us/step - loss: 0.8463 - mean_absolute_error: 0.4767 - acc: 0.7927 - val_loss: 0.8974 - val_mean_absolute_error: 0.4783 - val_acc: 0.7716
Epoch 3/25
210000/210000 [==============================] - 30s 142us/step - loss: 0.8423 - mean_absolute_error: 0.4755 - acc: 0.7807 - val_loss: 0.8943 - val_mean_absolute_error: 0.4778 - val_acc: 0.7661
Epoch 4/25
210000/210000 [==============================] - 30s 144us/step - loss: 0.8399 - mean_absolute_error: 0.4750 - acc: 0.7753 - val_loss: 0.8924 - val_mean_absolute_error: 0.4773 - val_acc: 0.7625
Epoch 5/25
210000/210000 [==============================] - 30s 145us/step - loss: 0.8383 - mean_absolute_error: 0.4746 - acc: 0.

/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Time: 66393.92 seconds

Time: 1106.57 minutes



### Calculate Metrics

In [12]:
metrics_df = du.create_autoencoder_metric_df(metrics, './autoencoder_metrics.csv')
metrics_df.describe()

,total,threshold,tpr_recall,precision,accuracy,f1_score,t_pos,f_pos,f_neg,t_neg,auc,time_to_fit
count,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000
mean,90000.0,9.969000,0.984408,0.593030,0.996707,0.740022,421.970000,289.700000,6.68000,89281.650000,0.990587,638.237242
std,0.0,0.532138,0.007026,0.016837,0.000193,0.013390,17.837909,17.328962,2.99454,25.809862,0.003506,70.354386
min,90000.0,9.000000,0.967366,0.547586,0.996244,0.706406,379.000000,255.000000,1.00000,89201.000000,0.982031,490.014900
25%,90000.0,9.600000,0.980121,0.580986,0.996556,0.730258,408.750000,278.750000,4.75000,89266.750000,0.988486,578.032481
50%,90000.0,9.800000,0.984340,0.592035,0.996739,0.739319,422.000000,286.500000,7.00000,89282.000000,0.990492,641.310576
75%,90000.0,10.400000,0.989349,0.606212,0.996836,0.749946,436.000000,301.750000,9.00000,89301.250000,0.993075,695.921592
max,90000.0,11.300000,0.997748,0.634670,0.997156,0.775832,461.000000,333.000000,14.00000,89340.000000,0.997450,756.114002
